In [13]:
%matplotlib notebook 
# Import modules
import os
import math
import numpy as np
import matplotlib.pyplot
from pyne import serpent
from pyne import nucname

ace_lib_path = '/home/andrei2/serpent/xsdata/endfb7'
acelib = "sss_endfb7.xsdata"
dep = serpent.parse_dep('../serpent/60-years-analysis/TMSR_dep.m', make_mats=False)
#res = serpent.parse_res('../serpent/60-years-analysis/TMSR_res.m')
#res2 = serpent.parse_res('/home/andrei2/Desktop/git/msr-neutronics/depletion/online_1200days_Pa_less_Th/less_Th_rate/core_res.m')

target_iso = 'U233'

plot_title = 'Pa outflow 1.25E-10, U-233 inflow 3.5E-9, Th 9E-9\n'
days = dep['DAYS']                        # Time array parsed from *_dep.m file
bu = dep['MAT_fuel_BURNUP']                        # Time array parsed from *_dep.m file
time_step = np.diff(days)                 # Depletion time step evaluation
names = dep['NAMES'][0].split()                      # Names of isotopes parsed from *_dep.m file
#keff_analytical = res['ANA_KEFF']         # K-eff parsing from dictionary
#keff_analytical2 = res2['ANA_KEFF']
#kinf_dict = res['ABS_KINF']               # K-inf parsing from dictionary
#kinf_dict2 = res2['ABS_KINF']
#bu = res['BURNUP']                        # Burnup parsing from dictionary
#keff_a = keff_analytical[:,0]             # K-eff value
#keff_error = keff_analytical[:,1]         # K-eff standart deviation
#kinf = kinf_dict[:,0]
#burnup = bu[:,0]
EOC = np.amax(days)                       # End of cycle (simulation time length)
#total_mass_list = dep['TOT_MASS']
adens_fuel = dep['MAT_fuel_ADENS']        # atomic density for each isotope in material 'fuel'
mdens_fuel = dep['MAT_fuel_MDENS']        # mass density for each isotope in material 'fuel'
vol_fuel = dep['MAT_fuel_VOLUME']         # total volume of material 'fuel'
density_fuel = mdens_fuel[-1,:]           # total mass density for material 'fuel'

print(dep.keys()) #prints keys     

['i70150', 'i581410', 'i511300', 'i942430', 'i581550', 'i962500', 'i611620', 'i340880', 'i641590', 'i591570', 'i601450', 'i531320', 'i320760', 'i391020', 'i481220', 'i511360', 'i401070', 'i511290', 'i621490', 'i330890', 'i471190', 'i461070', 'i461071', 'i902270', 'i320800', 'i340870', 'i390900', 'i601510', 'i651570', 'i310710', 'i531280', 'i441010', 'i471091', 'MAT_add_ING_TOX', 'i440970', 'i451160', 'i842120', 'i380860', 'i541310', 'i541311', 'MAT_pa_tank_ING_TOX', 'i370880', 'i411100', 'MAT_pa_tank_VOLUME', 'MAT_pa_tank_SF', 'i611490', 'i671670', 'i812100', 'i440990', 'i360930', 'i882280', 'i380880', 'i410991', 'i561450', 'i541390', 'i481120', 'i962390', 'i651640', 'i591470', 'i802060', 'i330800', 'MAT_pa_tank_A', 'i370950', 'i10010', 'i501231', 'i501230', 'i511310', 'i862200', 'i290730', 'i912341', 'i471120', 'i661580', 'i411030', 'i601580', 'i972490', 'i400940', 'i591580', 'i431030', 'i952460', 'i60150', 'i571450', 'MAT_add_BURNUP', 'i551420', 'i491141', 'i491140', 'i410940', 'i541

In [49]:
def get_library_isotopes():
    """ Returns the isotopes in the cross section library
    Parameters:
    -----------
    Returns:
    --------
    iso_array: array
        array of isotopes in cross section library:
    """
    lib_isos = []
    # check if environment variable is set
    if os.environ.get('SERPENT_DATA') is not None:
        path =  os.environ['SERPENT_DATA']
    else:
        path = ace_lib_path
    acelib_path = path +"/sss_endfb7.xsdata"
    
    with open(acelib_path, 'r') as f:
        metaflag = 0
        lines = f.readlines()
        for line in lines:
            iso = (line.split()[1]).split('.')[0]
            metaflag = line.split()[4]
            zzaaa = line.split()[3]
            if metaflag == '1':
                iso = iso.replace(iso, zzaaa)
            iso = str(iso) + str(metaflag)
            lib_isos.append(iso)
            
    return np.array(lib_isos)
    

#print (days)
print (days[21])

#print (bu)

#print(dep['iLOST'])
#print(adens_fuel[223-1])
#print(mdens_fuel[223-1])
print(nucname.serpent(952421))

#print(mdens_fuel[-1,:])

moment = -1

lib_isotopes = get_library_isotopes()

matf = open('mat_comp', 'w')
matf.write('mat fuel -%7.9E  tmp 900  vol %7.5E\n' % (density_fuel[moment],
                                                     vol_fuel[moment]))
for key, value in dep.items():
    if key[0]=='i':
        if key[1:] not in lib_isotopes:
            matf.write('%15s       -%7.9E\n' % (key[1:], mdens_fuel[value-1,moment]))
        elif key[1:] in lib_isotopes:
            matf.write('%11s.09c       -%7.9E\n' % (nucname.serpent(key[1:]), mdens_fuel[value-1,moment]))
        else:
            raise ValueError('Wrong isotope name')
        #print (mdens_fuel[value-2, 0])
        
matf.close()

7390.0
Am-242m
